# Property Price Guide Fetcher
This script allows you to find the price range for a given property listed on [Domain.com.au](https://www.domain.com.au/) where there is no price provided by the agent.
It does this by retrieving a property's details then performing a series of searches for different price ranges until it finds the upper and lower limits.

See [Medium post](https://medium.com/@alexdambra/how-to-get-aussie-property-price-guides-using-python-the-domain-api-afe871efac96).

*Developed by [Alex D'Ambra](https://www.linkedin.com/in/alexdambra/)*
*Updated by [Chris Bashall](https://bashallc.github.io/home/)*

---

Import required libraries

In [1]:
scarboro = 12678
doubleview = 4138



In [2]:
import json
import requests # this library is awesome: http://docs.python-requests.org/en/master/
import re, string, timeit
import time
import datetime
import csv

**Setup your parameters**

1.   Set your property ID. You can grab from end of the listing's URL.
      *eg. https://www.domain.com.au/132a-prince-edward-avenue-earlwood-nsw-2206-2014925785*
2.   Set your starting lower bound eg. 500k starting max price. The starting min price will default to your lower bound plus 400k, or you can set this manually. This will reduce the amount of API calls required.
3.   Set your increment value. This will increase/decrease the starting prices by this amount until a hit is made. eg. 50k. Smaller increments might be more accurate but increase API calls. Most agents would probably set guides in $50-100k increments one would assume anyway (can you tell I live in Sydney...)










In [4]:
# setup
property_id="2016187571"
starting_max_price=1000000
increment=10000
# when starting min price is zero we'll just use the lower bound plus 400k later on
starting_min_price=0

Provide your client credentials as per your [Domain](https://developer.domain.com.au) developer account.

Required: `client_id` and `client_secret`

Make a POST request to receive token.

In [5]:
# POST request for token
response = requests.post('https://auth.domain.com.au/v1/connect/token', data = {'client_id':'client_ccdf714ee11cc66cc1d679119502e8ed',"client_secret":"secret_14d43a0461bed7347d46dca18d201f34","grant_type":"client_credentials","scope":"api_listings_read","Content-Type":"text/json"})
token=response.json()
access_token=token["access_token"]
access_token

'6560ddafdbf408143f5e164541a07cdb'

Make a GET request to the listings endpoint to retrieve listing info for your selected property.

In [6]:
# GET Request for ID
url = "https://api.domain.com.au/v1/listings/"+property_id
auth = {"Authorization":"Bearer "+ access_token}
request = requests.get(url,headers=auth)
print(request)
r=request.json()

<Response [200]>


Extract property details

In [7]:
#get details
da=r['addressParts']
postcode=da['postcode']
suburb=da['suburb']
bathrooms=r['bathrooms']
bedrooms=r['bedrooms']
dateListed =r['dateListed']
date = datetime.datetime.today().strftime('%Y-%m-%d')
last_updated = r['dateUpdated']
agent = r['advertiserIdentifiers']

try:
    carspaces=r['carspaces']
except Exception:
    print("car spaces not provided")
    carspaces=-1
    pass

property_type=r['propertyTypes']
try:
    size=r['landAreaSqm']
except Exception:
    print("size not provided")
    size=-1
    pass

try:
    print(property_type,postcode, suburb, bedrooms, bathrooms,  carspaces)
except Exception:
    pass
# the below puts all relevant property types into a single string. eg. a property listing can be a 'house' and a 'townhouse'
n=0
property_type_str=""
for p in r['propertyTypes']:
  property_type_str=property_type_str+(r['propertyTypes'][int(n)])
  n=n+1
    
status = r['status']
lat = (r['geoLocation'].get('latitude')) 
long = (r['geoLocation'].get('longitude')) 
    


['house'] 6018 Gwelup 4.0 2.0 2.0


Now loop through a series of POST requests that search for your property starting with your starting max price, increasing by your increment each time until you get a result. 

We achieve this by using a `do while` loop. After receiving a response we put the list of property IDs into a Python list and then check if our original property_id is in that list. 

In [8]:
max_price=starting_max_price
searching_for_price=True

In [9]:
# Start your loop
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "maxPrice":max_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if int(property_id) in listings:
            max_price=max_price-increment
            print("Lower bound found: ", max_price)
            searching_for_price=False
    else:
        max_price=max_price+increment
        print("Not found. Increasing max price to ",max_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly  
        if max_price > 1200000:
            break

Lower bound found:  990000


Now do the same but from the upper end begining with your starting min price and decreasing by your increment. This will get us an upper bound.

In [10]:
searching_for_price=True
if starting_min_price>0:
  min_price=starting_min_price
else:  
  min_price=max_price+200000  

In [11]:
while searching_for_price:
    
    url = "https://api.domain.com.au/v1/listings/residential/_search" # Set destination URL here
    post_fields ={
      "listingType":"Sale",
        "minPrice":min_price,
        "pageSize":100,
      "propertyTypes":property_type,
      "minBedrooms":bedrooms,
        "maxBedrooms":bedrooms,
      "minBathrooms":bathrooms,
        "maxBathrooms":bathrooms,
      "locations":[
        {
          "state":"",
          "region":"",
          "area":"",
          "suburb":suburb,
          "postCode":postcode,
          "includeSurroundingSuburbs":False
        }
      ]
    }

    request = requests.post(url,headers=auth,json=post_fields)

    l=request.json()
    listings = []
    for listing in l:
        listings.append(listing["listing"]["id"])
    listings

    if int(property_id) in listings:
            min_price=min_price+increment
            print("Upper bound found: ", min_price)
            searching_for_price=False
    else:
        min_price=min_price-increment
        print("Not found. Decreasing min price to ",min_price)
        time.sleep(0.1)  # sleep a bit so you don't make too many API calls too quickly     
       

Not found. Decreasing min price to  1180000
Not found. Decreasing min price to  1170000
Not found. Decreasing min price to  1160000
Not found. Decreasing min price to  1150000
Not found. Decreasing min price to  1140000
Not found. Decreasing min price to  1130000
Not found. Decreasing min price to  1120000
Not found. Decreasing min price to  1110000
Not found. Decreasing min price to  1100000
Not found. Decreasing min price to  1090000
Not found. Decreasing min price to  1080000
Not found. Decreasing min price to  1070000
Not found. Decreasing min price to  1060000
Not found. Decreasing min price to  1050000
Not found. Decreasing min price to  1040000
Not found. Decreasing min price to  1030000
Not found. Decreasing min price to  1020000
Not found. Decreasing min price to  1010000
Not found. Decreasing min price to  1000000
Not found. Decreasing min price to  990000
Not found. Decreasing min price to  980000
Not found. Decreasing min price to  970000
Not found. Decreasing min price to 

Format your numbers for your final string.

In [12]:
if max_price<1000000:
  lower=max_price
  upper=min_price
  denom="k"
else: 
  lower=max_price/1000000
  upper=min_price/1000000
  denom="m"

Print your results!

In [13]:
# Print the results
print(da['displayAddress'])
print(r['headline'])
print("Property Type:",property_type_str)
print("Details: ",int(bedrooms),"bedroom,",int(bathrooms),"bathroom",int(carspaces),"carspace")
print("Display price:",r['priceDetails']['displayPrice'])      
if max_price==min_price:
  print("Price guide:","$",lower)
  print("$",int(lower/size))
else:
  print("Price range:","$",lower,"-","$",upper)
  print("$",int(lower/size)," - ","$",int(upper/size))
print("URL:",r['seoUrl'])
print("total size  :",size," sqm")


14 Lyndale Street, Gwelup WA 6018
SPACIOUS FAMILY HOME 4/5 BEDROOM 2 BATHROOM IN QUIET CUL-DE-SAC
Property Type: house
Details:  4 bedroom, 2 bathroom 2 carspace
Display price: $815,000
Price range: $ 990000 - $ 820000
$ 1210  -  $ 1002
URL: https://www.domain.com.au/14-lyndale-street-gwelup-wa-6018-2016187571
total size  : 818.0  sqm


In [14]:
## add to csv file




fields=[property_id,date,r['seoUrl'],da['displayAddress'],r['priceDetails']['displayPrice'],
        lower,upper,lower/size,upper/size,size,bedrooms,bathrooms,dateListed,last_updated,status,lat,long]
with open(r'results.csv', 'a') as f:
  writer = csv.writer(f)
  writer.writerow(fields)

In [15]:
####suburb details

In [16]:
r

{'objective': 'sale',
 'propertyTypes': ['house'],
 'status': 'live',
 'saleMode': 'buy',
 'channel': 'residential',
 'addressParts': {'stateAbbreviation': 'wa',
  'displayType': 'fullAddress',
  'streetNumber': '14',
  'street': 'Lyndale Street',
  'suburb': 'Gwelup',
  'postcode': '6018',
  'displayAddress': '14 Lyndale Street, Gwelup WA 6018'},
 'advertiserIdentifiers': {'advertiserType': 'agency',
  'advertiserId': 13233,
  'contactIds': [1266667],
  'agentIds': ['A1569']},
 'bathrooms': 2.0,
 'bedrooms': 4.0,
 'carspaces': 2.0,
 'dateUpdated': '2020-05-14T08:31:06.03Z',
 'dateListed': '2020-03-27T08:33:19Z',
 'description': 'Nestled in a quiet cul-de-sac this well built brick & tile home is perfectly located in the popular suburb of Gwelup\r\n\r\nAs you enter the home the property is divided into three separate levels, the entry level contains 4 bedrooms & the main bathroom, one excellent part of the third bedroom is that it is actually 2 bedrooms that are separated by a wall & do

In [17]:
# POST request for token
def suburb_details(suburb_id):
    response = requests.post('https://auth.domain.com.au/v1/connect/token', data = {'client_id':'client_ccdf714ee11cc66cc1d679119502e8ed',"client_secret":"secret_14d43a0461bed7347d46dca18d201f34","grant_type":"client_credentials","scope":"api_locations_read","Content-Type":"text/json"})
    token=response.json()
    access_token=token["access_token"]
    # GET Request for ID
    url = "https://api.domain.com.au/v1/locations/profiles/"+str(suburb_id)
    auth = {"Authorization":"Bearer "+ access_token}
    request = requests.get(url,headers=auth)
    print(request)
    r=request.json()
    return r
suburb_id = 12678
scarboro = 12678
doubleview = 4138

x = suburb_details(suburb_id)
x

<Response [200]>


{'domainLocationId': 12678,
 'postcode': '6019',
 'pfLocationId': 'WA3135',
 'surroundingSuburbs': [{'name': 'City Beach',
   'urlSlug': 'city-beach-wa-6015'},
  {'name': 'Karrinyup', 'urlSlug': 'karrinyup-wa-6018'},
  {'name': 'Doubleview', 'urlSlug': 'doubleview-wa-6018'},
  {'name': 'Trigg', 'urlSlug': 'trigg-wa-6029'},
  {'name': 'Wembley Downs', 'urlSlug': 'wembley-downs-wa-6019'}],
 'urlSlug': 'scarborough-wa-6019',
 'suburbName': 'Scarborough',
 'data': {'studiosForRent': 0,
  'terracedHousesForSale': 0,
  'semiDetachedHousesForSale': 1,
  'townhousesForRent': 13,
  'apartmentsAndUnitsForSale': 68,
  'apartmentsAndUnitsForRent': 37,
  'villasForSale': 22,
  'duplexesForSale': 1,
  'semiDetachedHousesForRent': 0,
  'studiosForSale': 0,
  'singlePercentage': 0.6473814711849125,
  'mostCommonAgeBracket': '20 to 39',
  'renterPercentage': 0.4204246087091275,
  'penthousesForSale': 0,
  'villasForRent': 13,
  'duplexesForRent': 3,
  'housesForSale': 51,
  'ownerOccupierPercentage': 0

In [18]:
data = x.get('data')
data.get('propertyCategories')[0].get('salesGrowthList')

[{'medianSoldPrice': 505000.0,
  'annualGrowth': 0.0,
  'numberSold': 124,
  'year': 2015},
 {'medianSoldPrice': 465000.0,
  'annualGrowth': -0.07920792079207921,
  'numberSold': 93,
  'year': 2016},
 {'medianSoldPrice': 448000.0,
  'annualGrowth': -0.03655913978494624,
  'numberSold': 83,
  'year': 2017},
 {'medianSoldPrice': 445000.0,
  'annualGrowth': -0.006696428571428571,
  'numberSold': 78,
  'year': 2018},
 {'medianSoldPrice': 450000.0,
  'annualGrowth': 0.011235955056179775,
  'numberSold': 92,
  'year': 2019},
 {'medianSoldPrice': 415000.0,
  'annualGrowth': -0.07777777777777778,
  'numberSold': 76,
  'year': 2020}]